In [1]:
import numpy as np
import torch
import json

from torch.utils.data import DataLoader

from collections import defaultdict

from GAIL.models.nets import Expert
from GAIL.models.gail import GAIL

from utilities import *
from utilitiesDL import *

In [2]:
DataFGMDir = '/project/iarpa/wifiHAR/HAR_survey/window_FGM/'
LSTMModelDir = './savedModels/selected/'

GAILModelJsonFileName = './GAIL/GAILModelConfig.json'
GAILModelJson = json.load(open(GAILModelJsonFileName))

GAILJsonFileName = './inputJson/GAIL/test.json'
GAILJson = json.load(open(GAILJsonFileName))

LSTMModelName = GAILJson['LSTMModelName']
noiseAmpRatio = GAILJson['noiseAmpRatio']
trDataRatio = GAILJson['trDataRatio']
trExpDataRatio = GAILJson['trExpDataRatio']
cudaID = GAILJson['cudaID']

torch.set_num_threads(1)

if cudaID >= 0:
    device = torch.device("cuda:"+str(cudaID))
    cudaAvbl = True
else:
    device = torch.device("cpu")
    cudaAvbl = False

dataType = LSTMModelName.split('_')[0]

if dataType == 'survey':
    fs = 1000 # 1 kHz
    nSubC = 30
    nRX = 3
    
    winLen = 1000
    thres = 60
    slideLen = 400
    activities = ['fall', 'pickup', 'run', 'sitdown', 'standup', 'walk']

LSTMType = LSTMModelName.split('_')[1]
bidirectional = (LSTMType == 'BLSTM')
nHidden = int(LSTMModelName.split('_')[3])
threshold = int(LSTMModelName.split('_')[5])
nLayer = int(LSTMModelName.split('_')[7])

# Load the LSTM model
HARNet = LSTMNet(nClasses=len(activities), input_size=nSubC*nRX, bidirectional=bidirectional,\
                hidden_size=nHidden, num_layers=1, seq_length=winLen//2, device=device)
HARNet.load_state_dict(torch.load(LSTMModelDir + LSTMModelName + '.cpkt'))
HARNet.to(device)

# Load dataset labelled with FGM attack
FGMdatasetDir = '/project/iarpa/wifiHAR/HAR_' + dataType + '/window_FGM/'
dataDict = {file:[] for file in activities}
tsDataDict = {file:[] for file in activities}

trExpDataset = list()
trAgentDataset = list()
# trDataset = list()
tsDataset = list()
for actInd, activity in enumerate(activities):
    dataDict[activity] = defaultdict(list)

    dataActFileName = FGMdatasetDir + LSTMModelName + '_' + activity + '.pt'
    dataAct = torch.load(dataActFileName)

    dataDict[activity]['obs'] =\
        torch.reshape(torch.squeeze(dataAct[0, :, :]), (-1, winLen//2, nSubC*nRX)).detach().cpu().numpy()
    dataDict[activity]['FGM'] = noiseAmpRatio *\
        torch.reshape(torch.squeeze(torch.squeeze(dataAct[1, :, :])),\
                      (-1, winLen//2, nSubC*nRX)).detach().cpu().numpy()
    dataDict[activity]['label'] =\
        actInd * np.ones((dataDict[activity]['obs'].shape[0]), dtype=int)

    datasetAct = FGMDataset(dataDict[activity], device)

    trExpDataset.append(torch.utils.data.Subset(datasetAct,\
                                                range(int(trDataRatio*trExpDataRatio*len(datasetAct)))))
    trAgentDataset.append(torch.utils.data.Subset(datasetAct,\
                                                  range(int(trDataRatio*trExpDataRatio*len(datasetAct)),\
                                                        int(trDataRatio*len(datasetAct)))))
    # trDataset.append(torch.utils.data.Subset(datasetAct,\
    #                                         range(int(trDataRatio*len(datasetAct)))))
    tsDataset.append(torch.utils.data.Subset(datasetAct,\
                                             range(int(trDataRatio*len(datasetAct)), len(datasetAct))))

    # print('activity:', activity, 'trDataset:', len(trDataset[-1]), 'tsDataset:', len(tsDataset[-1]))
    print('activity:', activity, 'trExpDataset:', len(trExpDataset[-1]),\
          'trAgentDataset:', len(trAgentDataset[-1]), 'tsDataset:', len(tsDataset[-1]))


trExpLoader = DataLoader(torch.utils.data.ConcatDataset(trExpDataset),\
                      batch_size=4, shuffle=True, generator=torch.Generator(device='cuda'))
trAgentLoader = DataLoader(torch.utils.data.ConcatDataset(trAgentDataset),\
                      batch_size=4, shuffle=True, generator=torch.Generator(device='cuda'))
# trLoader = DataLoader(torch.utils.data.ConcatDataset(trDataset),\
#                       batch_size=4, shuffle=True, generator=torch.Generator(device='cuda'))
tsLoader = DataLoader(torch.utils.data.ConcatDataset(tsDataset),\
                      batch_size=10, shuffle=True, generator=torch.Generator(device='cuda'))

# print('trExpLoader:', len(trLoader), 'tsLoader:', len(tsLoader))
print('trExpLoader:', len(trExpLoader), 'trAgentLoader', len(trAgentLoader), 'tsLoader:', len(tsLoader))

activity: fall trExpDataset: 93 trAgentDataset: 39 tsDataset: 311
activity: pickup trExpDataset: 103 trAgentDataset: 45 tsDataset: 347
activity: run trExpDataset: 253 trAgentDataset: 109 tsDataset: 847
activity: sitdown trExpDataset: 86 trAgentDataset: 37 tsDataset: 289
activity: standup trExpDataset: 64 trAgentDataset: 27 tsDataset: 214
activity: walk trExpDataset: 307 trAgentDataset: 132 tsDataset: 1026
trExpLoader: 227 trAgentLoader 98 tsLoader: 304


In [61]:
import numpy as np
import torch

from torch.nn import Module

from GAIL.models.nets import PolicyNetwork, ValueNetwork, Discriminator
from GAIL.utils.funcs import get_flat_grads, get_flat_params, set_params, \
    conjugate_gradient, rescale_and_linesearch

if torch.cuda.is_available():
    from torch.cuda import FloatTensor
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    from torch import FloatTensor


class GAIL(Module):
    def __init__(
        self,
        state_dim,
        action_dim,
        discrete,
        device,
        train_config=None
    ) -> None:
        super().__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.discrete = discrete
        self.device = device
        self.train_config = train_config

        self.pi = PolicyNetwork(self.state_dim, self.action_dim, self.discrete, self.device)
        self.v = ValueNetwork(self.state_dim, self.device)
        self.d = Discriminator(self.state_dim, self.action_dim, self.discrete, self.device)

    def get_networks(self):
        return [self.pi, self.v]

    def act(self, state):
        self.pi.eval()

        state = FloatTensor(state)
        distb = self.pi(state)

        action = distb.sample().detach().cpu().numpy()

        return action

    def train(self, HARNet, trExpLoader, trAgentLoader, tsLoader, render=False):
        num_iters = self.train_config["num_iters"]
        num_steps_per_iter = self.train_config["num_steps_per_iter"]
        horizon = self.train_config["horizon"]
        lambda_ = self.train_config["lambda"]
        gae_gamma = self.train_config["gae_gamma"]
        gae_lambda = self.train_config["gae_lambda"]
        eps = self.train_config["epsilon"]
        max_kl = self.train_config["max_kl"]
        cg_damping = self.train_config["cg_damping"]
        normalize_advantage = self.train_config["normalize_advantage"]

        opt_d = torch.optim.Adam(self.d.parameters())

        noiseAmpRatioList = [1e-4, 1e-3, 1e-2, 0.1]
        # noiseAmpRatioList = [1e-5]
        # noiseAmpRatioList = [1e-10]
        print('----White-box attack performance (Expert)----')
        lineBreakCount = 0
        print('[ampRatio, Acc.]:', end=' ')
        for noiseAmpRatio in noiseAmpRatioList:
            nData = 0.
            correct = 0.
            for tsBatch in tsLoader:
                pred_l,label_l = getPredsGAIL(tsBatch['obs'], tsBatch['FGM'], tsBatch['label'],\
                                              HARNet, noiseAmpRatio)
                nData += len(label_l)
                for pred, label in zip(pred_l, label_l):
                    correct += (pred == label)
                # accuracyList.append(correct/nData)
            print('[{0}, {1:.3f}]'.format(noiseAmpRatio, correct/nData), end=' ')
            lineBreakCount += 1
            if lineBreakCount == 4:
                print('')
                lineBreakCount = 0

        exp_obs = []
        exp_acts = []
        exp_acc_iter = []

        for trExpBatch in trExpLoader:
            exp_ob = trExpBatch['obs'].transpose(0, 1).reshape(-1, trExpBatch['obs'].shape[2])
            exp_act = trExpBatch['FGM'].transpose(0, 1).reshape(-1, trExpBatch['FGM'].shape[2])
            
            exp_obs.append(exp_ob)
            exp_acts.append(exp_act)

        for i in range(num_iters):
            if lineBreakCount != 0:
                print('')
            print('----Iteratons: {}----'.format(i))
            obs = []
            acts = []
            rets = []
            advs = []
            gms = []
            
            nData = 0
            correct = [0. for _ in noiseAmpRatioList]
            for trAgentBatch in trAgentLoader:
                nData += trAgentBatch['obs'].shape[0]
                seqLength = trAgentBatch['obs'].shape[1]
                # print('obs amp:', LA.norm(trAgentData['obs'].view(trAgentData['obs'].shape[0], -1), dim=1))
                obsAmp = LA.norm(trAgentBatch['obs'].view(trAgentBatch['obs'].shape[0], -1), dim=1)
                ob = trAgentBatch['obs'].transpose(0, 1).reshape(-1, trAgentBatch['obs'].shape[2])
                act = self.act(ob)
                actTorch = torch.reshape(torch.from_numpy(act), ([-1] + list(trAgentBatch['FGM'].shape[1:]))).to(device)
                
                lineBreakCount = 0
                for noiseAmpIndex, noiseAmpRatio in enumerate(noiseAmpRatioList):
                    # print(trAgentBatch['obs'].shape, actTorch.shape, trAgentBatch['FGM'].shape)
                    pred_l,label_l = getPredsGAIL(trAgentBatch['obs'], actTorch, trAgentBatch['label'],\
                                                    HARNet, noiseAmpRatio)

                    # print(pred_l, label_l)
                    for pred, label in zip(pred_l, label_l):
                        correct[noiseAmpIndex] += (pred == label)
                        # print(correct)
                    
                obs.append(ob)
                acts.append(act)

                for i, trAgentData in enumerate(trAgentBatch['obs']):
                    ep_obs = trAgentData
                    ep_acts = FloatTensor(act[i*seqLength : (i+1)*seqLength]).to(device)
                    ep_gms = torch.pow(gae_gamma, torch.arange(seqLength)).to(device)
                    ep_lmbs = torch.pow(gae_lambda, torch.arange(seqLength)).to(device)
                                        
                    ep_costs = (-1) * torch.log(self.d(ep_obs, ep_acts)).squeeze().detach()
                    ep_disc_costs = ep_gms * ep_costs
                    
                    ep_disc_rets = torch.flip(torch.flip(\
                        ep_disc_costs.to(device), dims=[0]).cumsum(dim=0), dims=[0])
                    # ep_disc_rets = FloatTensor([sum(ep_disc_costs[i:]) for i in range(seqLength)]).to(device)
                    ep_rets = ep_disc_rets / ep_gms
                    rets.append(ep_rets)

                    self.v.eval()
                    curr_vals = self.v(ep_obs).detach()
                    next_vals = torch.cat(
                        (self.v(ep_obs)[1:], FloatTensor([[0.]]).to(device))).detach()
                    ep_deltas = ep_costs.unsqueeze(-1) + gae_gamma * next_vals - curr_vals
                    ep_advs = FloatTensor([
                        ((ep_gms * ep_lmbs)[:seqLength - j].unsqueeze(-1) * ep_deltas[j:]).sum()
                        for j in range(seqLength)])

                    advs.append(ep_advs)
                    gms.append(ep_gms)

            # print('nData:', nData)
            print('[ampRatio, Acc.]:', end=' ')
            lineBreakCount = 0
            for noiseAmpIndex, noiseAmpRatio in enumerate(noiseAmpRatioList):
                print('[{0}, {1:.3f}]'.\
                        format(noiseAmpRatio, correct[noiseAmpIndex]/nData), end=' ')
                lineBreakCount += 1
                if lineBreakCount == 4:
                    print('')
                    lineBreakCount = 0

            # print('obs:', len(obs), 'acts:', len(acts))
                
                # rwd_iter_means.append(np.mean(rwd_iter))
                # print(
                #     "Iterations: {},   Accuracy: {}"
                #     .format(i + 1, np.mean(rwd_iter))
                # )

                    # print('trAgentData', trAgentData.shape)
                
                
            # exp_obs.append(trData['input'])
                        
        #     print(tsData['input'].shape, tsData['label'].shape)
        #     exp_obs.append(tsData['input'])
        #     exp_acts.append(tsData['label'])            
            # ep_rwds = FloatTensor(ep_rwds)

        # exp_rwd_mean = np.mean(exp_rwd_iter)
        # print(
        #     "Expert Reward Mean: {}".format(exp_rwd_mean)
        # )

        # print(exp_obs.shape, exp_acts.shape)
        # exp_obs = FloatTensor(np.array(exp_obs))
        # exp_acts = FloatTensor(np.array(exp_acts))
        # return exp_rwd_mean, rwd_iter_means

In [62]:
model = GAIL(state_dim=nSubC*nRX, action_dim=nSubC*nRX,\
             discrete=False, device=device, train_config=GAILModelJson)
model.train(HARNet, trExpLoader, trAgentLoader, tsLoader)

----White-box attack performance (Expert)----
[ampRatio, Acc.]: [1e-05, 0.971] 
----Iteratons: 0----
[ampRatio, Acc.]: [1e-05, 0.977] 
----Iteratons: 1----


KeyboardInterrupt: 